In [3]:


# %%python
from datetime import datetime, timedelta
from sqlmodel import SQLModel, create_engine, Session,select

# your model import (adjust the path as needed)
from fateforger.agents.schedular.models.calendar_event import (
    CalendarEvent,
    EventDateTime,
    CreatorOrganizer,
)

# 1) create an in-memory SQLite DB and reflect models
engine = create_engine("sqlite:///:memory:")
SQLModel.metadata.create_all(engine)

# 2) open a session, add a sample event, commit & refresh
with Session(engine) as session:
    ev = CalendarEvent(

        summary="🧪 Test Meeting",
        description="This is just a test insert",
        location="Virtual",
        creator=CreatorOrganizer(email="tester@example.com", display_name="Tester"),
        start=EventDateTime(date_time=datetime.utcnow(), time_zone="UTC"),
        end=EventDateTime(date_time=datetime.utcnow() + timedelta(hours=1), time_zone="UTC"),
    )
    session.add(ev)
    session.commit()
    session.refresh(ev)
    print("Inserted:", ev)

# 3) query all events and print
with Session(engine) as session:
    statement = select(CalendarEvent)
    all_events = session.exec(statement).all()
    print("All events in DB:", all_events)

Inserted: id=1 status=None description='This is just a test insert' color_id=None start=EventDateTime(date=None, date_time=datetime.datetime(2025, 7, 28, 16, 38, 6, 158768), time_zone='UTC') source=None extended_properties=None event_type=None calendar_id=None google_event_id=None summary='🧪 Test Meeting' location='Virtual' creator=CreatorOrganizer(id=None, email='tester@example.com', display_name='Tester', self_=None) end=EventDateTime(date=None, date_time=datetime.datetime(2025, 7, 28, 17, 38, 6, 158919), time_zone='UTC') transparency=None reminders=None
All events in DB: [CalendarEvent(id=1, status=None, description='This is just a test insert', color_id=None, start=EventDateTime(date=None, date_time=datetime.datetime(2025, 7, 28, 16, 38, 6, 158768), time_zone='UTC'), source=None, extended_properties=None, event_type=None, calendar_id=None, google_event_id=None, summary='🧪 Test Meeting', location='Virtual', creator=CreatorOrganizer(id=None, email='tester@example.com', display_name='

In [1]:
# %% cell
# 🔧 Test the DraftStore CRUD for ScheduleDraft

from datetime import date

# imports from your codebase (CalendarEvent, EventDateTime, CreatorOrganizer already imported)
from fateforger.agents.schedular.models.schedule_draft import (
    ScheduleDraft,
    DraftStore,
)

# 1. Spin up an in-memory SQLite and create tables
engine = create_engine("sqlite:///:memory:")
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    store = DraftStore(session)

    # 2. Build two sample CalendarEvent objects
    now = datetime.utcnow()
    ev1 = CalendarEvent(
        summary="Draft Event 1",
        start=EventDateTime(date_time=now, time_zone="UTC"),
        end=EventDateTime(date_time=now + timedelta(hours=1), time_zone="UTC"),
        creator=CreatorOrganizer(email="user1@example.com", display_name="User One"),
    )
    ev2 = CalendarEvent(
        summary="Draft Event 2",
        start=EventDateTime(date_time=now + timedelta(hours=2), time_zone="UTC"),
        end=EventDateTime(date_time=now + timedelta(hours=3), time_zone="UTC"),
        creator=CreatorOrganizer(email="user2@example.com", display_name="User Two"),
    )

    # 3. Create & save a ScheduleDraft for today
    today = date.today()
    draft = ScheduleDraft(date=today, events=[ev1, ev2])
    saved = store.save(draft)
    print("✅ Saved draft:", saved)

    # 4. Retrieve it by date
    loaded = store.get_by_date(today)
    print("🔍 Loaded draft:", loaded)

    # 5. List all drafts
    all_drafts = store.list_all()
    print("📑 All drafts in DB:", all_drafts)

    # 6. Delete and verify removal
    store.delete(saved.id)
    print("🗑 After delete, drafts:", store.list_all())

PydanticUserError: Error when building FieldInfo from annotated attribute. Make sure you don't have any field name clashing with a type annotation.

For further information visit https://errors.pydantic.dev/2.11/u/unevaluable-type-annotation